In [1]:
import torch
import numpy as np
import os
from EduNLP.Pretrain import DisenQTokenizer, train_disenQNet
from EduNLP.Vector import DisenQModel, T2V
from EduNLP.I2V import DisenQ
from EduNLP.ModelZoo import load_items

os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")

D:\MySoftwares\Anaconda\envs\data\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# 训练自己的disenQNet模型
## 1. 数据

In [6]:
BASE_DIR = "E:\Workustc\EduNLP\workMaster\EduNLP"

data_dir = f"{BASE_DIR}/tests/test_vec/test_data"
output_dir = f"{BASE_DIR}/examples/test_model/data"

disen_data_train = load_items(f"{data_dir}/disenq_train.json")
disen_data_test = load_items(f"{data_dir}/disenq_test.json")


## 2. 训练和评估

In [8]:
train_params = {
    'epoch': 50,
    'batch': 256,
    'trim_min': 5,
    'max_len': 250,
    "hidden": 128,
    'warm_up': 5,
    "device": "cuda"
}

train_disenQNet(
    disen_data_train,
    output_dir,
    output_dir,
    train_params=train_params,
    test_items=disen_data_test,
)

save vocab to E:\Workustc\EduNLP\workMaster\EduNLP/examples/test_model/data\vocab.list
save concept to E:\Workustc\EduNLP\workMaster\EduNLP/examples/test_model/data\concept.list
save word2vec to E:\Workustc\EduNLP\workMaster\EduNLP/examples/test_model/data\wv.th
processing raw data for QuestionDataset...
vocab size: 3702
concept size: 5
load vocab from E:\Workustc\EduNLP\workMaster\EduNLP/examples/test_model/data\vocab.list
load concept from E:\Workustc\EduNLP\workMaster\EduNLP/examples/test_model/data\concept.list
load word2vec from E:\Workustc\EduNLP\workMaster\EduNLP/examples/test_model/data\wv.th
processing raw data for QuestionDataset...
Start training the disenQNet...
[Epoch  1] train loss: 1.7298
[Epoch  2] train loss: 1.5702
[Epoch  3] train loss: 1.4829
[Epoch  4] train loss: 1.4349
[Epoch  5] train loss: 1.3843
[Epoch  6] train loss: 1.5143, eval loss: 1.5105
[Epoch  7] train loss: 1.4440, eval loss: 1.4501
[Epoch  8] train loss: 1.3966, eval loss: 1.4040
[Epoch  9] train los

## 3.使用模型

In [10]:
tokenizer_kwargs = {
    "tokenizer_config_dir": output_dir,
}
i2v = DisenQ('disenQ', 'disenq', output_dir, tokenizer_kwargs=tokenizer_kwargs, device="cuda")

test_items = [
    {"content": "10 米 的 (2/5) = 多少 米 的 (1/2),有 公 式"},
    {"content": "10 米 的 (2/5) = 多少 米 的 (1/2),有 公 式 , 如 图 , 若 $x,y$ 满 足 约 束 条 件 公 式"},
]

t_vec = i2v.infer_token_vector([test_items[1]], key=lambda x: x["content"])
i_vec = i2v.infer_item_vector(test_items[1], key=lambda x: x["content"], vector_type="k")

print(i_vec.shape) # == torch.Size([x, 128])
print(t_vec.shape) # == torch.Size([x, 150, 128])


torch.Size([1, 128])
torch.Size([1, 23, 128])
